In [1]:
from atd_utils.data_utils import CfbdClient
import pandas as pd
import numpy as np

In [2]:
cfbdc = CfbdClient()

In [3]:
cfbdc.help()

Methods:
	conn
	hit_endpoint
	is_table
	load_to_df
	pull_data
	pull_year
	save_data


Attributes:
	cfbd_configuration
	data_dir


True

In [4]:
# test = cfbdc.pull_data([1869, 2022], 'get_games')

In [8]:
import requests

In [9]:
header = 
test = requests.get("https://api.collegefootballdata.com/games?year=2022&week=1&seasonType=regular")

In [35]:
from os.path import join
def save(data_dir: str, sub_dir: str, filename: str, data: str) -> None:
    path = join(data_dir, sub_dir)
    if not os.path.exists(path):
        os.makedirs(path)
    with open(f"{path}/{filename}", "w") as f:
        f.write(data)

In [38]:
url = f"https://api.collegefootballdata.com/games/players?year={2022}&week={6}&seasonType={'postseason'}"
print(f'Year: {year}, week: {week}, season_type: {season_type}')
response = requests.get(url, headers=headers)

Year: 2016, week: 2, season_type: regular


In [41]:
if json.loads(response.text):
    print(1)

In [45]:
import requests
from requests.exceptions import RequestException
from retrying import retry

# Define the maximum number of retries and the backoff interval between retries
MAX_RETRIES = 3
BACKOFF_FACTOR = 2

@retry(wait_exponential_multiplier=BACKOFF_FACTOR * 1000, stop_max_attempt_number=MAX_RETRIES)
def get_response(url, headers):
    try:
        # Call the API function
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Raise an exception for non-2xx responses
    except RequestException as e:
        # Raise a custom exception for API errors
        raise Exception(f"API Error: {e} !!!!!\n{url} FAILED!!!!!!!!")
    return response


In [1]:
import os
import requests

api_key = os.environ["CFBD_API_KEY"]
headers = {"Authorization": f"Bearer {api_key}"}
for year in range(2022, 1800, -1):
    for season_type in ['regular', 'postseason']:
        for week in range(1, 17):
            url = f"https://api.collegefootballdata.com/games/players?year={year}&week={week}&seasonType={season_type}"
            print(f'Year: {year}, week: {week}, season_type: {season_type}')
            response = get_response(url, headers)
            if json.loads(response.text):
                save('../data/', 'get_player_game_stats', f'get_player_game_stats_{year}_{season_type}_{week}.json', response.text)

In [30]:
# import sys
# print(sys.pwd())

AttributeError: module 'sys' has no attribute 'pwd'

In [37]:
from os.path import join
import os
data_dir = '../../atd_utils/data'
sub_dir = 'get_player_game_stats'
path = join(data_dir, sub_dir)
new_data_dir = '../../atd_analysis/data'
new_path = join(new_data_dir, sub_dir)
for file in os.listdir(path):
    if file[0] == "." or file[0] == "O":
        continue
    # print(file)
    with open(join(path, file), "r") as f:
        data = f.read()
    words = file.split('.')[0].split('_')
    front = "_".join(words[:-2])
    week = words[-2]
    season_type = words[-1]
    new_filename = f"{front}_{season_type}_{week}.json"
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    with open(join(new_path, new_filename), "w") as f:
        f.write(data)
        

In [6]:
def extract_year(data, year, week, season_type):
    rows = []
    for game in data:
        game_id = game['id']
        teams = game['teams']
        for team in teams:
            categories = team['categories']
            if 'home_away' in team.keys():
                home_away = team['home_away']
                team_name = team['school']['name']
                conference = team['school']['conference']
            else:
                home_away = team['homeAway']
                team_name = team['school']
                conference = team['conference']
            categories = team['categories']
            for category in categories:
                name = category['name']
                types = category['types']
                for typ in types:
                    typ_name = typ['name']
                    athletes = typ['athletes']
                    for athlete in athletes:
                        athlete_id = athlete['id']
                        athlete_name = athlete['name']
                        stat = athlete['stat']
                        row = {
                            'year': year,
                            'week': week,
                            'season_type': season_type,
                            'game_id': game_id,
                            'home_away': 'home' if home_away else 'away',
                            'team_name': team_name,
                            'conference': conference,
                            'category': name,
                            'type': typ_name,
                            'athlete_id': athlete_id,
                            'athlete_name': athlete_name,
                            'stat': stat
                        }
                        rows.append(row)
    return rows

In [7]:
from os.path import join
import os
import json

sub_dir = 'get_player_game_stats'
data_dir = '../../atd_analysis/data'
path = join(new_data_dir, sub_dir)
results = []
for file in os.listdir(path):
    if file[0] == "." or file[0] == "O":
        continue
    # print(file)
    with open(join(path, file), "r") as f:
        data = json.loads(f.read())
    words = file.split('.')[0].split('_')
    week = words[-1]
    season_type = words[-2]
    year = words[-3]
    results += extract_year(data, year, week, season_type)
        

In [8]:
len(results)

16509667

In [9]:
get_player_game_stats = pd.DataFrame(results)

In [10]:
get_player_game_stats.shape

(16509667, 12)

In [12]:
import sqlite3
conn = sqlite3.connect(join(data_dir, "cfbd.db"))

In [13]:
get_player_game_stats.to_sql( "get_player_game_stats", conn, if_exists="replace")

16509667

In [14]:
data_dir

'../../atd_analysis/data'

In [24]:
import os
import requests

api_key = os.environ["CFBD_API_KEY"]
headers = {"Authorization": f"Bearer {api_key}"}
url = "https://api.collegefootballdata.com/games/players?year=2022&week=1&seasonType=regular"

print(time.strftime("%H:%M:%S", time.localtime()))
start_time = time.time()
response = requests.get(url, headers=headers)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time)

01:17:39
Elapsed time: 15.606133937835693


In [21]:
import json
len(json.loads(response.text))

145

In [23]:
# json.loads(response.text)[0]

In [13]:
print(restextponse)

<Response [200]>


In [2]:
# from __future__ import print_function
# import time
# import cfbd
# from cfbd.rest import ApiException
# import os
# from pprint import pprint

# # Configure API key authorization: ApiKeyAuth
# configuration = cfbd.Configuration()
# configuration.api_key['Authorization'] = os.environ["CFBD_API_KEY"]
# configuration.api_key_prefix['Authorization'] = 'Bearer'

# # create an instance of the API class
# api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
# year = 2022 # int | Year/season filter for games
# week = 1 # int | Week filter (optional)
# # team = "Air Force" # int | Week filter (optional)
# season_type = 'regular' # str | Season type filter (regular or postseason) (optional) (default to regular)

# try:
# # Player game stats
#     start_time = time.time()
#     print(time.strftime("%H:%M:%S", time.localtime()))
#     api_response = api_instance.get_player_game_stats(year, week=week, season_type=season_type) #, team=team)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Elapsed time:", elapsed_time)
#     pprint(api_response)
# except ApiException as e:
#     print("Exception when calling GamesApi->get_player_game_stats: %s\n" % e)

In [ ]:
cfbdc.pull_data([1800, 2022], 'get_player_game_stats')

Pulling 2022.
	Pulled 0 records 
                    from get_player_game_stats.


  0%|                                                                                                                                                                                                                                | 0/16 [00:00<?, ?it/s]

{'year': 2022, 'season_type': 'regular', 'week': 1}


In [ ]:
cfbdc.help()

In [ ]:
# cfbdc.pull_data([1800, 1892], 'get_fbs_teams') #, save=False, force=True)

In [5]:
# test = cfbdc.load_to_df(, save_to_db=True)

In [ ]:
done = False


In [86]:
rows = extract_data(test)
df = pd.DataFrame(rows)
# df = df[['category', 'type', 'athlete_id', 'athlete_name', 'stat']]
print(df.head())

     game_id home_away team_name conference category   type  athlete_id   
0  401403858      home      None       None  punting   LONG     4799994  \
1  401403858      home      None       None  punting  In 20     4799994   
2  401403858      home      None       None  punting     TB     4799994   
3  401403858      home      None       None  punting    AVG     4799994   
4  401403858      home      None       None  punting    YDS     4799994   

    athlete_name  stat  
0  Brett Thorson    53  
1  Brett Thorson     1  
2  Brett Thorson     0  
3  Brett Thorson  53.0  
4  Brett Thorson    53  


In [85]:
test[0]

{'id': 401403858,
 'teams': [{'school': {'name': None, 'conference': None},
   'home_away': True,
   'points': 49,
   'categories': [{'name': 'punting',
     'types': [{'name': 'LONG',
       'athletes': [{'id': 4799994, 'name': 'Brett Thorson', 'stat': '53'}]},
      {'name': 'In 20',
       'athletes': [{'id': 4799994, 'name': 'Brett Thorson', 'stat': '1'}]},
      {'name': 'TB',
       'athletes': [{'id': 4799994, 'name': 'Brett Thorson', 'stat': '0'}]},
      {'name': 'AVG',
       'athletes': [{'id': 4799994, 'name': 'Brett Thorson', 'stat': '53.0'}]},
      {'name': 'YDS',
       'athletes': [{'id': 4799994, 'name': 'Brett Thorson', 'stat': '53'}]},
      {'name': 'NO',
       'athletes': [{'id': 4799994, 'name': 'Brett Thorson', 'stat': '1'}]}]},
    {'name': 'kicking',
     'types': [{'name': 'PTS',
       'athletes': [{'id': 4379410, 'name': 'Jack Podlesny', 'stat': '7'}]},
      {'name': 'XP',
       'athletes': [{'id': 4379410, 'name': 'Jack Podlesny', 'stat': '7/7'}]},
    

In [82]:
pd.json_normalize(test)

,id,teams
0,401403858,"[{'school': {'name': None, 'conference': None}..."


In [7]:
test['season_year']

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
97104    1965
97105    1965
97106    1965
97107    1965
97108    1965
Name: season_year, Length: 97109, dtype: int64

In [13]:
test.start_date.dtype

dtype('O')

In [14]:
test['season_year']

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
97104    1965
97105    1965
97106    1965
97107    1965
97108    1965
Name: season_year, Length: 97109, dtype: int64

In [16]:
test[(test.home_team == "Georgia") & (test.away_team == "TCU")][['season_year','game_date']]

,season_year,game_date
6533,2022,2023-01-10 00:30:00+00:00
20965,1988,1988-09-10 00:00:00+00:00
46245,1941,1942-01-01 00:00:00+00:00
71389,1980,1980-09-27 00:00:00+00:00


In [16]:
pd.to_datetime(test.start_date).dt.month

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
95458    1965
95459    1965
95460    1965
95461    1965
95462    1965
Name: start_date, Length: 95463, dtype: int32

In [23]:
get_coaches = cfbdc.load_to_df('get_coaches')

In [24]:
get_coaches.head()

,school,year,games,wins,losses,ties,preseason_rank,postseason_rank,srs,sp_overall,sp_offense,sp_defense,first_name,last_name,hire_date
0,Toledo,2008,12,3,9,0,NaN,NaN,-14.7,-11.4,22.6,33.1,Tom,Amstutz,None
1,Nevada,2008,13,7,6,0,NaN,NaN,0.3,0.9,34.0,33.0,Chris,Ault,None
2,Rice,2008,13,10,3,0,NaN,NaN,1.6,4.8,40.5,35.1,David,Bailiff,None
3,Virginia Tech,2008,14,10,4,0,17.0,15.0,7.8,8.1,20.6,13.1,Frank,Beamer,None
4,Oregon,2008,13,10,3,0,21.0,10.0,15.6,19.6,38.4,18.3,Mike,Bellotti,None


In [49]:
q = """
WITH games AS (
    SELECT 
        id AS game_id,
        home_team AS team_name,
        home_conference AS conference,
        CASE 
            WHEN home_points > away_points THEN 'W' 
            WHEN home_points = away_points THEN 'T' 
            ELSE 'L' END AS win_lose,
        game_date, 
        season_year AS season,
        season_type
    FROM get_games 
    WHERE season_year > 1950
    AND (notes IS NULL OR (LOWER(notes) NOT LIKE '%championship%'
    AND LOWER(notes) NOT LIKE '%bowl%'))
    
    UNION ALL
    
    SELECT 
        id AS game_id,
        away_team AS team_name,
        away_conference AS conference,
        CASE 
            WHEN home_points > away_points THEN 'L' 
            WHEN home_points = away_points THEN 'T' 
            ELSE 'W' END AS win_lose,
        game_date, 
        season_year AS season,
        season_type
    FROM get_games 
    WHERE season_year > 1950
    AND (notes IS NULL OR (LOWER(notes) NOT LIKE '%championship%'
    AND LOWER(notes) NOT LIKE '%bowl%'))
)

, unblemished_regular_seasons AS (

    SELECT
        team_name,
        season,
        conference
    FROM games
    WHERE season_type = 'regular'
    GROUP BY 1, 2, 3
    HAVING SUM(CASE WHEN win_lose != 'L' THEN 1 END) = COUNT(1)
    
    )

, consecutives AS (
    WITH recursive consecutive_years AS (
        SELECT
            team_name,
            season,
            conference,
            1 AS cumulative_seasons
        FROM unblemished_regular_seasons
        UNION ALL
        SELECT
            a.team_name,
            a.season,
            a.conference,
            b.cumulative_seasons + 1 AS cumulative_seasons
        FROM unblemished_regular_seasons AS a
        JOIN consecutive_years AS b ON a.team_name = b.team_name
        AND a.season = b.season + 1
    )
    SELECT
        team_name,
        season,
        conference,
        cumulative_seasons
    FROM consecutive_years
    
)
SELECT
    team_name,
    season,
    conference,
    cumulative_seasons
FROM consecutives
WHERE conference IN 
('FBS Independents',
 'SEC',
 'Big Ten',
 'Southern',
 'ACC',
 'Mid-American',
 'Southwest',
 'Big 12',
 'Conference USA',
 'Pac-10',
 'Pacific',
 'Mountain West',
 'Big 8',
 'Sun Belt',
 'Big Sky',
 'CAA',
 'Pac-12',
 'Big East',
 'American Athletic',
 'Big 6',
 'Pac-8',
 'Big 7')
 AND team_name = "Georgia"
ORDER BY 4 DESC, 1 DESC

"""

pd.read_sql_query(q, cfbdc.conn).head(30)

,team_name,season,conference,cumulative_seasons
0,Georgia,1968,SEC,1
1,Georgia,1980,SEC,1
2,Georgia,1982,SEC,1
3,Georgia,2022,SEC,1


In [58]:
q = """


    SELECT 
        notes
    FROM get_games 
    WHERE season_year = 2021
    AND "Georgia" in (home_team, away_team)
    ORDER BY game_date
"""

print(pd.read_sql_query(q, cfbdc.conn).head(30))

                                           notes
0                                           None
1                                           None
2                                           None
3                                           None
4                                           None
5                                           None
6                                           None
7                                           None
8                                           None
9                                           None
10                                          None
11                                          None
12                                          None
13  CFP Semifinal at the Capital One Orange Bowl
14       CFP National Championship pres. by AT&T


In [53]:
q = """
WITH games AS (
    SELECT 
        id AS game_id,
        home_team AS team_name,
        home_conference AS conference,
        CASE 
            WHEN home_points > away_points THEN 'W' 
            WHEN home_points = away_points THEN 'T' 
            ELSE 'L' END AS win_lose,
        game_date, 
        season_year AS season,
        season_type,
        game_date,
        notes
    FROM get_games 
    WHERE season_year > 1950
    AND season_type = 'regular'
    AND (notes IS NULL OR (LOWER(notes) NOT LIKE '%championship%'
    AND LOWER(notes) NOT LIKE '%bowl%'))
    
    UNION ALL
    
    SELECT 
        id AS game_id,
        away_team AS team_name,
        away_conference AS conference,
        CASE 
            WHEN home_points > away_points THEN 'L' 
            WHEN home_points = away_points THEN 'T' 
            ELSE 'W' END AS win_lose,
        game_date, 
        season_year AS season,
        season_type,
        game_date,
        notes
    FROM get_games 
    WHERE season_year > 1950
    AND season_type = 'regular'
    AND (notes IS NULL OR (LOWER(notes) NOT LIKE '%championship%'
    AND LOWER(notes) NOT LIKE '%bowl%'))
)
, unblemished_regular_seasons AS (

    SELECT
        team_name,
        season,
        conference
    FROM games
    GROUP BY 1, 2, 3
    HAVING SUM(CASE WHEN win_lose != 'L' THEN 1 END) = COUNT(1)
    
    )

SELECT
    *
FROM games
WHERE team_name = "Georgia"
AND season = 2021
ORDER BY game_date DESC

"""
"""
, consecutives AS (
    WITH recursive consecutive_years AS (
        SELECT
            team_name,
            season,
            conference,
            1 AS cumulative_seasons
        FROM unblemished_regular_seasons
        UNION ALL
        SELECT
            a.team_name,
            a.season,
            a.conference,
            b.cumulative_seasons + 1 AS cumulative_seasons
        FROM unblemished_regular_seasons AS a
        JOIN consecutive_years AS b ON a.team_name = b.team_name
        AND a.season = b.season + 1
    )
    SELECT
        team_name,
        season,
        conference,
        cumulative_seasons
    FROM consecutive_years
    
)
SELECT
    team_name,
    season,
    conference,
    cumulative_seasons
FROM consecutives
WHERE conference IN 
('FBS Independents',
 'SEC',
 'Big Ten',
 'Southern',
 'ACC',
 'Mid-American',
 'Southwest',
 'Big 12',
 'Conference USA',
 'Pac-10',
 'Pacific',
 'Mountain West',
 'Big 8',
 'Sun Belt',
 'Big Sky',
 'CAA',
 'Pac-12',
 'Big East',
 'American Athletic',
 'Big 6',
 'Pac-8',
 'Big 7')
 AND team_name = "Georgia"
ORDER BY 4 DESC, 1 DESC
"""

pd.read_sql_query(q, cfbdc.conn).head(30)

,game_id,team_name,conference,win_lose,game_date,season,season_type,game_date:1,notes
0,401282154.0,Georgia,SEC,L,2021-12-04 21:00:00+00:00,2021,regular,2021-12-04 21:00:00+00:00,None
1,401282148.0,Georgia,SEC,W,2021-11-27 17:00:00+00:00,2021,regular,2021-11-27 17:00:00+00:00,None
2,401282136.0,Georgia,SEC,W,2021-11-20 17:00:00+00:00,2021,regular,2021-11-20 17:00:00+00:00,None
3,401282133.0,Georgia,SEC,W,2021-11-13 20:30:00+00:00,2021,regular,2021-11-13 20:30:00+00:00,None
4,401282122.0,Georgia,SEC,W,2021-11-06 16:00:00+00:00,2021,regular,2021-11-06 16:00:00+00:00,None
5,401282117.0,Georgia,SEC,W,2021-10-30 19:30:00+00:00,2021,regular,2021-10-30 19:30:00+00:00,None
6,401282105.0,Georgia,SEC,W,2021-10-16 19:30:00+00:00,2021,regular,2021-10-16 19:30:00+00:00,None
7,401282097.0,Georgia,SEC,W,2021-10-09 19:30:00+00:00,2021,regular,2021-10-09 19:30:00+00:00,None
8,401282090.0,Georgia,SEC,W,2021-10-02 16:00:00+00:00,2021,regular,2021-10-02 16:00:00+00:00,None
9,401282088.0,Georgia,SEC,W,2021-09-25 16:00:00+00:00,2021,regular,2021-09-25 16:00:00+00:00,None


In [25]:
q = """
WITH games AS (
    SELECT 
        id AS game_id,
        home_team AS team_name,
        home_conference AS conference,
        CASE 
            WHEN home_points > away_points THEN 'W' 
            WHEN home_points = away_points THEN 'T' 
            ELSE 'L' END AS win_lose,
        game_date, 
        season_year AS season,
        season_type
    FROM get_games 
    WHERE season_year > 1900
    
    UNION ALL
    
    SELECT 
        id AS game_id,
        away_team AS team_name,
        away_conference AS conference,
        CASE 
            WHEN home_points > away_points THEN 'L' 
            WHEN home_points = away_points THEN 'T' 
            ELSE 'W' END AS win_lose,
        game_date, 
        season_year AS season,
        season_type
    FROM get_games 
    WHERE season_year > 2010
)
SELECT
    conference,
    SUM(CASE WHEN win_lose = 'W' THEN 1 END)
FROM games
GROUP BY 1
ORDER BY 2 DESC
"""

pd.read_sql_query(q, cfbdc.conn).conference.tolist()

['FBS Independents',
 'SEC',
 'Big Ten',
 'Southern',
 'ACC',
 'Mid-American',
 'Southwest',
 None,
 'Big 12',
 'Western Athletic',
 'Western',
 'Missouri Valley',
 'Conference USA',
 'Pac-10',
 'Pacific',
 'Ivy',
 'Mountain West',
 'Big 8',
 'MVFC',
 'Sun Belt',
 'Big Sky',
 'CAA',
 'Pac-12',
 'Southland',
 'Pioneer',
 'Rocky Mountain',
 'Big East',
 'MEAC',
 'American Athletic',
 'SWAC',
 'OVC',
 'Border',
 'NEC',
 'Patriot',
 'Big South',
 'PCAA',
 'Big 6',
 'Skyline',
 'Big West',
 'Pac-8',
 'Big 7',
 'FCS Independents',
 'Pennsylvania State Athletic',
 'AAWU',
 'Atlantic 10',
 'Northern Sun',
 'Mountain State',
 'Mid America',
 'Great American',
 'SIAC',
 'Mountain East',
 'Great West',
 'South Atlantic',
 'Gulf South',
 'Lone Star',
 'CIAA',
 'GLIAC',
 'Great Midwest Athletic',
 'Northeast 10',
 'Great Lakes',
 'Mid Atlantic',
 'Ohio',
 'Midwest',
 "Presidents'",
 'Minnesota',
 'Centennial',
 'American Southwest',
 'AWC',
 'CCIW',
 'Michigan',
 'North Coast',
 'NACC',
 'Metro Atl

In [33]:
test[0]['season_type']

'regular'

In [26]:
for col in test.columns:
    print(col)

id
season
week
season_type
start_time_tbd
completed
neutral_site
conference_game
attendance
venue_id
venue
home_id
home_team
home_conference
home_division
home_points
home_post_win_prob
home_pregame_elo
home_postgame_elo
away_id
away_team
away_conference
away_division
away_points
away_post_win_prob
away_pregame_elo
away_postgame_elo
excitement_index
highlights
notes
year
home_q1_points
home_q2_points
home_q3_points
home_q4_points
home_ot_total_points
home_ot_points_per_period
home_num_ot_periods
away_q1_points
away_q2_points
away_q3_points
away_q4_points
away_ot_total_points
away_ot_points_per_period
away_num_ot_periods
game_date
